In [0]:
#all spark imports
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

#instantiate the spark session
spark = SparkSession.builder.appName("Movie-Prediction").getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", 2)

In [0]:
from pyspark.ml.feature import Imputer, StringIndexer, VectorAssembler
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
# File location and type
path = "/FileStore/tables/movies.csv"

df = spark.read \
  .format("csv") \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("path", path) \
  .load()

In [0]:
display(df)

movieId,rating,userId
2,3,0
3,1,0
5,2,0
9,4,0
11,1,0
12,2,0
15,1,0
17,1,0
19,1,0
21,1,0


In [0]:
df.printSchema()

root
-- movieId: integer (nullable = true)
-- rating: integer (nullable = true)
-- userId: integer (nullable = true)

In [0]:
# check the count of null values for each column
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 0| 0| 0|
+-------+------+------+

In [0]:
(training, test) = df.randomSplit([0.8, 0.2])

In [0]:
(training2, test2) = df.randomSplit([0.7, 0.3])

In [0]:
als = ALS(maxIter=5, regParam=0.05, rank= 15, userCol= "userId", itemCol= "movieId", ratingCol="rating", coldStartStrategy = "drop")

In [0]:
eval = RegressionEvaluator(metricName= "rmse", labelCol= "rating",predictionCol= "prediction")
eval2 = RegressionEvaluator(metricName= "mse", labelCol= "rating",predictionCol= "prediction")
eval3 = RegressionEvaluator(metricName= "mae", labelCol= "rating",predictionCol= "prediction")

In [0]:
model = als.fit(training)

In [0]:
predictions = model.transform(test)

In [0]:
rmse = eval.evaluate(predictions)
mse = eval2.evaluate(predictions)
mae = eval3.evaluate(predictions)

In [0]:
print("rmse-") 
print(rmse) 
print("mse-")
print(mse)
print("mae-")
print(mae) 

rmse-
1.1501724845428571
mse-
1.322896744199489
mae-
0.7908617407083511

In [0]:
model2 = als.fit(training2)

In [0]:
predictions2 = model2.transform(test2)

In [0]:
rmse2 = eval.evaluate(predictions2)
mse2 = eval2.evaluate(predictions2)
mae2 = eval3.evaluate(predictions2)

In [0]:
print("rmse-") 
print(rmse2) 
print("mse-")
print(mse2)
print("mae-")
print(mae2) 

rmse-
1.259408289740908
mse-
1.5861092402681192
mae-
0.8795168348219209

In [0]:
parameters = (ParamGridBuilder()
             .addGrid(als.regParam, [0.01, 0.5, 1])
             .addGrid(als.maxIter, [5, 10, 20])
             .addGrid(als.rank, [5, 10, 20])
             .build())

In [0]:
als = ALS( userCol= "userId", itemCol= "movieId", ratingCol="rating", coldStartStrategy = "drop")

In [0]:
cv = CrossValidator(estimator=als, estimatorParamMaps=parameters, evaluator=eval, numFolds=2)

In [0]:
model = cv.fit(training)

In [0]:
best_model = model.bestModel

In [0]:
print("**Best Model**")
# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

**Best Model**
 Rank: 10
 MaxIter: 10
 RegParam: 0.1

In [0]:
final_predictions = best_model.transform(test)

In [0]:
RMSE = eval.evaluate(final_predictions)
print(RMSE)

1.1031829802818578

In [0]:
 df.select("movieId").distinct().count()

Out[39]: 100

In [0]:
# Generate n Recommendations for all users
recommendations = best_model.recommendForAllUsers(100)
recommendations.show()

+------+--------------------+
userId| recommendations|
+------+--------------------+
 10|[[2, 3.2510662], ...|
 0|[[2, 2.752071], [...|
 20|[[22, 3.801374], ...|
 1|[[62, 3.0596368],...|
 11|[[18, 4.673748], ...|
 21|[[53, 4.1254935],...|
 2|[[8, 4.506863], [...|
 12|[[46, 4.8220625],...|
 22|[[75, 4.6854963],...|
 13|[[93, 2.9517941],...|
 3|[[30, 4.2171826],...|
 23|[[55, 4.8598413],...|
 4|[[52, 3.2882385],...|
 14|[[52, 4.8325915],...|
 24|[[52, 4.5451274],...|
 5|[[55, 3.8853645],...|
 25|[[33, 3.0501833],...|
 15|[[46, 4.1023855],...|
 6|[[25, 3.929694], ...|
 16|[[51, 4.068044], ...|
+------+--------------------+
only showing top 20 rows

In [0]:
nrecommendations = recommendations.withColumn("rec_exp", explode("recommendations")).select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))
nrecommendations.limit(10).show()

+------+-------+---------+
userId|movieId| rating|
+------+-------+---------+
 10| 2|3.2510662|
 10| 49|2.9934607|
 10| 40|2.9191973|
 10| 25|2.6260495|
 10| 87|2.5806065|
 10| 9|2.5066047|
 10| 81|2.4661276|
 10| 89|2.4410143|
 10| 4|2.2875152|
 10| 0| 2.239838|
+------+-------+---------+

In [0]:
user11_rec=nrecommendations.filter('userId = 11')
user11_rec.show()

+------+-------+---------+
userId|movieId| rating|
+------+-------+---------+
 11| 18| 4.673748|
 11| 32| 4.607893|
 11| 30|4.5838795|
 11| 23|4.4727707|
 11| 79|4.3102436|
 11| 48|4.1468024|
 11| 27|3.7176304|
 11| 19|3.6555498|
 11| 38|3.6257546|
 11| 13|3.5872586|
 11| 66|3.5589588|
 11| 90|3.5564613|
 11| 81| 3.468621|
 11| 8| 3.24898|
 11| 55| 3.179845|
 11| 49|3.1142466|
 11| 80| 3.092257|
 11| 33|2.9068248|
 11| 83|2.8998706|
 11| 71| 2.758349|
+------+-------+---------+
only showing top 20 rows

In [0]:
user11_exist=df.filter('userId = 11')
user11_exist.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 0| 1| 11|
 6| 2| 11|
 9| 1| 11|
 10| 1| 11|
 11| 1| 11|
 12| 1| 11|
 13| 4| 11|
 16| 1| 11|
 18| 5| 11|
 19| 4| 11|
 20| 1| 11|
 21| 1| 11|
 22| 1| 11|
 23| 5| 11|
 25| 1| 11|
 27| 5| 11|
 30| 5| 11|
 32| 5| 11|
 35| 3| 11|
 36| 2| 11|
+-------+------+------+
only showing top 20 rows

In [0]:
user11_rec.join(user11_exist, ['movieId'], 'left_anti').show(15, False)

+-------+------+---------+
movieId|userId|rating |
+-------+------+---------+
8 |11 |3.24898 |
55 |11 |3.179845 |
49 |11 |3.1142466|
33 |11 |2.9068248|
83 |11 |2.8998706|
46 |11 |2.7517567|
24 |11 |2.401533 |
7 |11 |2.401338 |
44 |11 |2.3883357|
73 |11 |2.3654099|
65 |11 |2.2733686|
34 |11 |2.2519782|
68 |11 |2.074239 |
91 |11 |1.9261204|
4 |11 |1.8661505|
+-------+------+---------+
only showing top 15 rows

In [0]:
user23_rec=nrecommendations.filter('userId = 23')
user23_rec.show()

+------+-------+---------+
userId|movieId| rating|
+------+-------+---------+
 23| 55|4.8598413|
 23| 32| 4.801601|
 23| 27| 4.703674|
 23| 49|4.5659018|
 23| 48| 4.267776|
 23| 90| 4.1931|
 23| 46| 4.114995|
 23| 65|4.0326576|
 23| 64|3.7455106|
 23| 50| 3.68508|
 23| 18|3.6748781|
 23| 30|3.6058986|
 23| 13|3.5325122|
 23| 23|3.5203753|
 23| 17|3.4165099|
 23| 10|3.1975677|
 23| 20|3.1566741|
 23| 94|3.1401424|
 23| 7| 3.045062|
 23| 73|2.8444695|
+------+-------+---------+
only showing top 20 rows

In [0]:
user23_exist=df.filter('userId = 23')
user23_exist.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 0| 1| 23|
 2| 1| 23|
 4| 1| 23|
 6| 2| 23|
 10| 4| 23|
 12| 1| 23|
 13| 4| 23|
 14| 1| 23|
 15| 1| 23|
 18| 4| 23|
 22| 2| 23|
 23| 4| 23|
 24| 1| 23|
 25| 1| 23|
 26| 1| 23|
 27| 5| 23|
 28| 1| 23|
 29| 1| 23|
 30| 4| 23|
 32| 5| 23|
+-------+------+------+
only showing top 20 rows

In [0]:
user23_rec.join(user23_exist, ['movieId'], 'left_anti').show(15, False)

+-------+------+---------+
movieId|userId|rating |
+-------+------+---------+
90 |23 |4.1931 |
46 |23 |4.114995 |
17 |23 |3.4165099|
20 |23 |3.1566741|
94 |23 |3.1401424|
7 |23 |3.045062 |
16 |23 |2.6539147|
79 |23 |2.6131525|
80 |23 |2.5549629|
8 |23 |2.4708216|
52 |23 |2.2473412|
31 |23 |2.2421894|
91 |23 |2.2203598|
81 |23 |2.134837 |
40 |23 |2.1043615|
+-------+------+---------+
only showing top 15 rows

In [0]:
movie_avg=df.groupBy("movieId").agg(F.mean('rating'), F.count('rating'))

In [0]:
movie_avg.sort(col("count(rating)").desc()).limit(10).show()

+-------+------------------+-------------+
movieId| avg(rating)|count(rating)|
+-------+------------------+-------------+
 22| 2.05| 20|
 50| 1.8| 20|
 29| 2.4| 20|
 51| 2.0| 20|
 6| 1.45| 20|
 2|2.1052631578947367| 19|
 15|1.1578947368421053| 19|
 55|1.7894736842105263| 19|
 68|2.1578947368421053| 19|
 94| 2.473684210526316| 19|
+-------+------------------+-------------+

In [0]:
user_avg=df.groupBy("userId").agg(F.mean('rating'), F.count('rating'))

In [0]:
user_avg.sort(col("count(rating)").desc()).limit(10).show()

+------+------------------+-------------+
userId| avg(rating)|count(rating)|
+------+------------------+-------------+
 14|1.7894736842105263| 57|
 6|1.4385964912280702| 57|
 22|2.1607142857142856| 56|
 11|2.2857142857142856| 56|
 4|1.5636363636363637| 55|
 12|1.8545454545454545| 55|
 7|1.6296296296296295| 54|
 9|1.7924528301886793| 53|
 18|1.7307692307692308| 52|
 23|2.1346153846153846| 52|
+------+------------------+-------------+